In [135]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy
from lime import lime_tabular
import string
import nltk
from nltk.corpus import stopwords
import random
from sklearn.naive_bayes import MultinomialNB


In [136]:
# Data Preprocessing

def preprocessing(fake=pd.read_csv("True.csv"),true=pd.read_csv("True.csv")):
    lenfake=len(fake)
    lentrue=len(true)
    fake['target'] = 'fake'
    true['target'] = 'true'
    data = pd.concat([fake, true]).reset_index(drop = True)
    data.drop(["date"],axis=1,inplace=True)
    data.drop(["title"],axis=1,inplace=True)
    data['text'] = data['text'].apply(lambda x: x.lower())

    def punctuation_removal(text):
        all_list = [char for char in text if char not in string.punctuation]
        clean_str = ''.join(all_list)
        return clean_str

    data['text'] = data['text'].apply(punctuation_removal)
    nltk.download('stopwords')
    stop = stopwords.words('english')
    data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    print('data lenght = ',len(data))
    print('lenfake = ',lenfake)
    print('lentrue = ',lentrue)
    return(data)


In [137]:
def dataproc(data,nbfake,nbtrue):
    print(data)
    if nbfake>=(lenfake-1):
        print('Not enough fake news in dataset')
    if nbtrue>=(lentrue-1):
        print('Not enough true news in dataset')

    a=random.sample(range(0, lenfake), nbfake)
    b=random.sample(range(lenfake, (lenfake+lentrue-1)), nbtrue)
    print(b)
    d=data.iloc[a][:]
    e=data.iloc[b][:]
    data2=pd.concat([d,e], axis=0)

    X_train,X_test,y_train,y_test = train_test_split(data2['text'],data2.target,test_size=0.2,random_state=42)

    return(X_train,X_test,y_train,y_test)

#data2=dataproc(preprocessing(),nbfake,nbtrue)
#print(data2[0])


In [138]:
'''def subjects(data):
    print(data.groupby(['subject'])['text'].count())
    data.groupby(['subject'])['text'].count().plot(kind="bar")
    plt.show()

subjects(dataproc(data2,13,14))'''

'def subjects(data):\n    print(data.groupby([\'subject\'])[\'text\'].count())\n    data.groupby([\'subject\'])[\'text\'].count().plot(kind="bar")\n    plt.show()\n\nsubjects(dataproc(data2,13,14))'

In [139]:
def vecto(X_train,X_test):
    
    X_t=pd.concat([X_train,X_test])
    print('X_t = ',X_t.shape)
    ltotal=len(X_t)
    print(X_t)
    vectorizer = TfidfVectorizer(lowercase=False)
    v = vectorizer.fit_transform(X_t)
    '''
    print('X_t = ',X_t)
    print('X_t type = ',type(X_t))
    print('X_t shape = ', X_t.shape)
    print('X_t[4] = ',X_t[4])
    print('X_t[4] type = ',type(X_t[4]))
    #print('v = ',v)
    print('v shape = ', v.shape)
'''
    features=vectorizer.get_feature_names_out()
    ltrain=len(X_train)
    vtrain=v[0:ltrain][:]
    vtest=v[ltrain:][:]
    vtrain=scipy.sparse.csr_matrix.toarray(vtrain)
    vtest=scipy.sparse.csr_matrix.toarray(vtest)
    '''
    print(type(vtest))
    print('vtest = ',vtest)
    print('vtest shape = ',vtest.shape)
    print('FEATURES = ',features)
    print('length features = ', len(features))
'''
    return(vtrain,vtest,vectorizer)
    

print(vecto(data2[0],data2[1]))

X_t =  (10,)
37422    washington reuters taliban increased amount te...
18043    rome reuters italy traditional political parti...
24297    beijing reuters chinese president xi jinping u...
17888    tokyo reuters japan yuriko koike says went pol...
34466    mexico city reuters mexican soldiers arbitrari...
7432     jakartaislamabadcairo reuters many muslims aro...
18671    washington reuters us president donald trump e...
28637    beijing reuters whether presidentelect donald ...
29723    reuters lifesized nude statue us republican pr...
20846    cape town reuters south africa top prosecutor ...
Name: text, dtype: object
(array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.03395049, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
   

In [140]:

def classifLogReg(vtrain,vtest,y_train,y_test):
    mo=LogisticRegression()
    mo.fit(vtrain,y_train)
    m=mo.predict(vtest)
    m=np.array(m)
    y_test=np.array(y_test)
    print(y_test)
    print(m)

    print("accuracy: {}%".format(round(accuracy_score(y_test, m)*100,2)))
    return(round(accuracy_score(y_test, m)*100,2))
print(classifLogReg)

<function classifLogReg at 0x7fbcb09714c0>


In [141]:
'''
def p(news):
    print('news = ',news)
    print("news type = ",type(news))
    news=pd.Series(news)
    v3=vectorizer.transform(news)

    print('v3 = ',v3)
    print('fonction p lancée')

    v3=scipy.sparse.csr_matrix.toarray(v3)
    print('v3 shape = ',v3.shape)
    r3=mo.predict(v3)
    print('r3 = ',r3)
    return(r3)


#xk=p(['test'])
#print(X_train[2].split(" "))
#x=p(X_train[2].split(" "))
x=p(X_train[2])
#print(X_train[2])
#print(np.shape(x))
#print((X_train[2].split(" ")))
'''


'\ndef p(news):\n    print(\'news = \',news)\n    print("news type = ",type(news))\n    news=pd.Series(news)\n    v3=vectorizer.transform(news)\n\n    print(\'v3 = \',v3)\n    print(\'fonction p lancée\')\n\n    v3=scipy.sparse.csr_matrix.toarray(v3)\n    print(\'v3 shape = \',v3.shape)\n    r3=mo.predict(v3)\n    print(\'r3 = \',r3)\n    return(r3)\n\n\n#xk=p([\'test\'])\n#print(X_train[2].split(" "))\n#x=p(X_train[2].split(" "))\nx=p(X_train[2])\n#print(X_train[2])\n#print(np.shape(x))\n#print((X_train[2].split(" ")))\n'

In [147]:

def classifNaiveBayes(vtrain,y_train,vectorizer,X_test):
    mo=MultinomialNB()
    mo.fit(vtrain,y_train)

    print(type(X_test))

    #print('X_test = ',X_test)
    #print('X_test type = ',type(X_test))
    print('xtest index = ',X_test[:][0])
    i=X_test[:][0] 
    j=i[2]
    o=pd.Series(X_test[j])
    o=vectorizer.transform(o)
    o=scipy.sparse.csr_matrix.toarray(o)
    o=o[0]
    '''
    print('o = ', o)
    print('o type = ',type(o))
    print('o shape = ',o.shape)


    print('vtrain shape = ',vtrain.shape)
    print('vtrain type = ',type(vtrain))
    print('vtrain[5][5] = ',vtrain[5][5])
    print('vtrain[5][5] type = ',type(vtrain[5][5]))
    '''
    explainer = lime_tabular.LimeTabularExplainer(vtrain,mode="classification",categorical_features=features,feature_names=features)
    exp=explainer.explain_instance(data_row=o,predict_fn=mo.predict_proba, num_features=20) #labels=features
    exp.as_pyplot_figure()
    from matplotlib import pyplot as plt
    plt.tight_layout()
    exp.show_in_notebook()

In [143]:
'''print(features)
with plt.style.context("ggplot"):
    fig = plt.figure(figsize=(8,6))
    plt.barh(range(len(mo.coef_[0])), mo.coef_[0], color=["red" if coef<0 else "green" for coef in mo.coef_[0]])
    plt.yticks(range(len(mo.coef_[0])), features);
    plt.title("Weights")
    '''

'print(features)\nwith plt.style.context("ggplot"):\n    fig = plt.figure(figsize=(8,6))\n    plt.barh(range(len(mo.coef_[0])), mo.coef_[0], color=["red" if coef<0 else "green" for coef in mo.coef_[0]])\n    plt.yticks(range(len(mo.coef_[0])), features);\n    plt.title("Weights")\n    '

In [148]:
def limenaivebayes(nbfake,nbtrue):
    data=preprocessing()
    x1=dataproc(data,nbfake,nbtrue)
    X_train=x1[0]
    X_test=x1[1]
    y_train=x1[2]
    y_test=x1[3]
    vtrain,vtest,vectorizer=vecto(X_train,X_test)
    classifNaiveBayes(vtrain,y_train,vectorizer,X_test)
    
limenaivebayes(5,5)



[nltk_data] Downloading package stopwords to /Users/evan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


data lenght =  42834
lenfake =  21417
lentrue =  21417
                                                    text       subject target
0      washington reuters head conservative republica...  politicsNews   fake
1      washington reuters transgender people allowed ...  politicsNews   fake
2      washington reuters special counsel investigati...  politicsNews   fake
3      washington reuters trump campaign adviser geor...  politicsNews   fake
4      seattlewashington reuters president donald tru...  politicsNews   fake
...                                                  ...           ...    ...
42829  brussels reuters nato allies tuesday welcomed ...     worldnews   true
42830  london reuters lexisnexis provider legal regul...     worldnews   true
42831  minsk reuters shadow disused sovietera factori...     worldnews   true
42832  moscow reuters vatican secretary state cardina...     worldnews   true
42833  jakarta reuters indonesia buy 11 sukhoi fighte...     worldnews   true

[42834 r

KeyError: 0